In [1]:
import pandas as pd

## Explanation of inputs and outputs

### Sample Rules

> These are the sample rules that will be the output from agent 1.

| Scheme | Scheme Rules |
|--------|--------------|
| **Unemployment Assistance** | 1. Resident of the State. 2. Unemployed but capable of and available for work. 3. Have insufficient means. |
| **Old Age Assistance** | 1. Resident of the State. 2. Aged 66 years or over. 3. Have insufficient means. |
| **Blind Pension** | 1. Resident of the State. 2. Aged 18 years or over. 3. Have a vision impairment. 4. Have insufficient means. |
| **Widow’s and Widower’s (Non-Contributory) Pension** | 1. Resident of the State. 2. Widow or widower not cohabiting. 3. Have insufficient means. |
| **Deserted Wife’s Allowance** | 1. Resident of the State. 2. Deserted by husband. 3. Have insufficient means. 4. Not cohabiting. |
| **Carer's Allowance** | 1. Resident of the State. 2. Providing full-time care and attention to a person in need. 3. Have insufficient means. |
| **Prisoner's Wife Allowance** | 1. Resident of the State. 2. Husband in prison. 3. Have insufficient means. 4. Not cohabiting. |
| **Single Woman’s Allowance** | 1. Resident of the State. 2. Aged 58 years or over. 3. Single woman living alone. 4. Have insufficient means. |
| **Orphan’s (Non-Contributory) Pension** | 1. Resident of the State. 2. Under 18 years of age. 3. Orphaned. 4. Have insufficient means. |

### Sample Output Schema

>This is a table schema with all the columns needed for discerning the scheme. It will be the output of agent 2.

| Field Name                | Data Type | Constraints                                      | Description                                                   |
|---------------------------|-----------|--------------------------------------------------|---------------------------------------------------------------|
| `id`                      | INT       | PRIMARY KEY, AUTO_INCREMENT                      | Unique identifier for each person                             |
| `first_name`              | VARCHAR   | NOT NULL                                         | First name of the person                                      |
| `last_name`               | VARCHAR   | NOT NULL                                         | Last name of the person                                       |
| `age`                     | INT       | NOT NULL, CHECK (age >= 0)                       | Age of the person                                             |
| `is_resident`             | BOOLEAN   | NOT NULL                                         | Whether the person is a resident of the state                 |
| `is_unemployed`           | BOOLEAN   | NOT NULL                                         | Whether the person is unemployed                              |
| `has_insufficient_means`  | BOOLEAN   | NOT NULL                                         | Whether the person has insufficient means                     |
| `vision_impaired`         | BOOLEAN   | NOT NULL                                         | Whether the person has a vision impairment                    |
| `is_widow_or_widower`     | BOOLEAN   | NOT NULL                                         | Whether the person is a widow or widower                      |
| `is_not_cohabiting`       | BOOLEAN   | NOT NULL                                         | Whether the person is not cohabiting                          |
| `is_deserted_by_husband`  | BOOLEAN   | NOT NULL                                         | Whether the person is deserted by their husband               |
| `is_providing_care`       | BOOLEAN   | NOT NULL                                         | Whether the person is providing full-time care to someone     |
| `husband_in_prison`       | BOOLEAN   | NOT NULL                                         | Whether the person's husband is in prison                     |
| `is_single_woman`         | BOOLEAN   | NOT NULL                                         | Whether the person is a single woman                          |
| `is_orphan`               | BOOLEAN   | NOT NULL                                         | Whether the person is an orphan                               |

### Sample Input Schema

> This is a schema with some irrelevant information that is not required to determine the scheme. This is the input to agent 2.

| Field Name                | Data Type | Constraints                                      | Description                                                   |
|---------------------------|-----------|--------------------------------------------------|---------------------------------------------------------------|
| `id`                      | INT       | PRIMARY KEY, AUTO_INCREMENT                      | Unique identifier for each person                             |
| `first_name`              | VARCHAR   | NOT NULL                                         | First name of the person                                      |
| `last_name`               | VARCHAR   | NOT NULL                                         | Last name of the person                                       |
| `age`                     | INT       | NOT NULL, CHECK (age >= 0)                       | Age of the person                                             |
| `is_resident`             | BOOLEAN   | NOT NULL                                         | Whether the person is a resident of the state                 |
| `is_unemployed`           | BOOLEAN   | NOT NULL                                         | Whether the person is unemployed                              |
| `has_insufficient_means`  | BOOLEAN   | NOT NULL                                         | Whether the person has insufficient means                     |
| `vision_impaired`         | BOOLEAN   | NOT NULL                                         | Whether the person has a vision impairment                    |
| `is_widow_or_widower`     | BOOLEAN   | NOT NULL                                         | Whether the person is a widow or widower                      |
| `is_not_cohabiting`       | BOOLEAN   | NOT NULL                                         | Whether the person is not cohabiting                          |
| `is_deserted_by_husband`  | BOOLEAN   | NOT NULL                                         | Whether the person is deserted by their husband               |
| `is_providing_care`       | BOOLEAN   | NOT NULL                                         | Whether the person is providing full-time care to someone     |
| `husband_in_prison`       | BOOLEAN   | NOT NULL                                         | Whether the person's husband is in prison                     |
| `is_single_woman`         | BOOLEAN   | NOT NULL                                         | Whether the person is a single woman                          |
| `is_orphan`               | BOOLEAN   | NOT NULL                                         | Whether the person is an orphan                               |
| `department`              | VARCHAR   | NOT NULL                                         | Department the person belongs to                              |
| `hiring_manager`          | VARCHAR   | NOT NULL                                         | Hiring manager for the person                                 |
| `number_of_children`      | INT       | NOT NULL, CHECK (number_of_children >= 0)        | Number of children the person has                             |
| `date_of_hire`            | DATE      |                                                  | Date the person was hired                                     |
| `employment_status`       | VARCHAR   | NOT NULL                                         | Employment status of the person                               |
| `annual_income`           | DECIMAL   | CHECK (annual_income >= 0)                       | Annual income of the person                                   |

## Agent Code Development

In [2]:
import json
from openai import OpenAI

In [3]:
client = OpenAI()

In [4]:
rules = """
| Scheme | Scheme Rules |
|--------|--------------|
| **Unemployment Assistance** | 1. Resident of the State. 2. Unemployed but capable of and available for work. 3. Have insufficient means. |
| **Old Age Assistance** | 1. Resident of the State. 2. Aged 66 years or over. 3. Have insufficient means. |
| **Blind Pension** | 1. Resident of the State. 2. Aged 18 years or over. 3. Have a vision impairment. 4. Have insufficient means. |
| **Widow’s and Widower’s (Non-Contributory) Pension** | 1. Resident of the State. 2. Widow or widower not cohabiting. 3. Have insufficient means. |
| **Deserted Wife’s Allowance** | 1. Resident of the State. 2. Deserted by husband. 3. Have insufficient means. 4. Not cohabiting. |
| **Carer's Allowance** | 1. Resident of the State. 2. Providing full-time care and attention to a person in need. 3. Have insufficient means. |
| **Prisoner's Wife Allowance** | 1. Resident of the State. 2. Husband in prison. 3. Have insufficient means. 4. Not cohabiting. |
| **Single Woman’s Allowance** | 1. Resident of the State. 2. Aged 58 years or over. 3. Single woman living alone. 4. Have insufficient means. |
| **Orphan’s (Non-Contributory) Pension** | 1. Resident of the State. 2. Under 18 years of age. 3. Orphaned. 4. Have insufficient means. |
"""

In [5]:
input_schema = {
  "columns": [
    {
      "name": "id",
      "type": "INT",
      "constraints": ["PRIMARY KEY", "AUTO_INCREMENT"],
      "description": "Unique identifier for each person"
    },
    {
      "name": "first_name",
      "type": "VARCHAR",
      "constraints": ["NOT NULL"],
      "description": "First name of the person"
    },
    {
      "name": "last_name",
      "type": "VARCHAR",
      "constraints": ["NOT NULL"],
      "description": "Last name of the person"
    },
    {
      "name": "age",
      "type": "INT",
      "constraints": ["NOT NULL", "CHECK (age >= 0)"],
      "description": "Age of the person"
    },
    {
      "name": "is_resident",
      "type": "BOOLEAN",
      "constraints": ["NOT NULL"],
      "description": "Whether the person is a resident of the state"
    },
    {
      "name": "is_unemployed",
      "type": "BOOLEAN",
      "constraints": ["NOT NULL"],
      "description": "Whether the person is unemployed"
    },
    {
      "name": "has_insufficient_means",
      "type": "BOOLEAN",
      "constraints": ["NOT NULL"],
      "description": "Whether the person has insufficient means"
    },
    {
      "name": "vision_impaired",
      "type": "BOOLEAN",
      "constraints": ["NOT NULL"],
      "description": "Whether the person has a vision impairment"
    },
    {
      "name": "is_widow_or_widower",
      "type": "BOOLEAN",
      "constraints": ["NOT NULL"],
      "description": "Whether the person is a widow or widower"
    },
    {
      "name": "is_not_cohabiting",
      "type": "BOOLEAN",
      "constraints": ["NOT NULL"],
      "description": "Whether the person is not cohabiting"
    },
    {
      "name": "is_deserted_by_husband",
      "type": "BOOLEAN",
      "constraints": ["NOT NULL"],
      "description": "Whether the person is deserted by their husband"
    },
    {
      "name": "is_providing_care",
      "type": "BOOLEAN",
      "constraints": ["NOT NULL"],
      "description": "Whether the person is providing full-time care to someone"
    },
    {
      "name": "husband_in_prison",
      "type": "BOOLEAN",
      "constraints": ["NOT NULL"],
      "description": "Whether the person's husband is in prison"
    },
    {
      "name": "is_single_woman",
      "type": "BOOLEAN",
      "constraints": ["NOT NULL"],
      "description": "Whether the person is a single woman"
    },
    {
      "name": "is_orphan",
      "type": "BOOLEAN",
      "constraints": ["NOT NULL"],
      "description": "Whether the person is an orphan"
    },
    {
      "name": "department",
      "type": "VARCHAR",
      "constraints": ["NOT NULL"],
      "description": "Department the person belongs to"
    },
    {
      "name": "hiring_manager",
      "type": "VARCHAR",
      "constraints": ["NOT NULL"],
      "description": "Hiring manager for the person"
    },
    {
      "name": "number_of_children",
      "type": "INT",
      "constraints": ["NOT NULL", "CHECK (number_of_children >= 0)"],
      "description": "Number of children the person has"
    },
    {
      "name": "date_of_hire",
      "type": "DATE",
      "constraints": [],
      "description": "Date the person was hired"
    },
    {
      "name": "employment_status",
      "type": "VARCHAR",
      "constraints": ["NOT NULL"],
      "description": "Employment status of the person"
    },
    {
      "name": "annual_income",
      "type": "DECIMAL",
      "constraints": ["CHECK (annual_income >= 0)"],
      "description": "Annual income of the person"
    }
  ]
}

In [6]:
expected_output_schema = {
  "columns": [
    {
      "name": "id",
      "type": "INT",
      "constraints": ["PRIMARY KEY", "AUTO_INCREMENT"],
      "description": "Unique identifier for each person"
    },
    {
      "name": "first_name",
      "type": "VARCHAR",
      "constraints": ["NOT NULL"],
      "description": "First name of the person"
    },
    {
      "name": "last_name",
      "type": "VARCHAR",
      "constraints": ["NOT NULL"],
      "description": "Last name of the person"
    },
    {
      "name": "age",
      "type": "INT",
      "constraints": ["NOT NULL", "CHECK (age >= 0)"],
      "description": "Age of the person"
    },
    {
      "name": "is_resident",
      "type": "BOOLEAN",
      "constraints": ["NOT NULL"],
      "description": "Whether the person is a resident of the state"
    },
    {
      "name": "is_unemployed",
      "type": "BOOLEAN",
      "constraints": ["NOT NULL"],
      "description": "Whether the person is unemployed"
    },
    {
      "name": "has_insufficient_means",
      "type": "BOOLEAN",
      "constraints": ["NOT NULL"],
      "description": "Whether the person has insufficient means"
    },
    {
      "name": "vision_impaired",
      "type": "BOOLEAN",
      "constraints": ["NOT NULL"],
      "description": "Whether the person has a vision impairment"
    },
    {
      "name": "is_widow_or_widower",
      "type": "BOOLEAN",
      "constraints": ["NOT NULL"],
      "description": "Whether the person is a widow or widower"
    },
    {
      "name": "is_not_cohabiting",
      "type": "BOOLEAN",
      "constraints": ["NOT NULL"],
      "description": "Whether the person is not cohabiting"
    },
    {
      "name": "is_deserted_by_husband",
      "type": "BOOLEAN",
      "constraints": ["NOT NULL"],
      "description": "Whether the person is deserted by their husband"
    },
    {
      "name": "is_providing_care",
      "type": "BOOLEAN",
      "constraints": ["NOT NULL"],
      "description": "Whether the person is providing full-time care to someone"
    },
    {
      "name": "husband_in_prison",
      "type": "BOOLEAN",
      "constraints": ["NOT NULL"],
      "description": "Whether the person's husband is in prison"
    },
    {
      "name": "is_single_woman",
      "type": "BOOLEAN",
      "constraints": ["NOT NULL"],
      "description": "Whether the person is a single woman"
    },
    {
      "name": "is_orphan",
      "type": "BOOLEAN",
      "constraints": ["NOT NULL"],
      "description": "Whether the person is an orphan"
    }
  ]
}

In [7]:
completion = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a schema editing assistant, capable of extracting only the relevant columns based on a task. You only output in JSON format. Do not include other text or markdown tags in the output."},
        {"role": "user", "content": f"The relevant task is to determine the scheme of a person based on the following rules: {rules}"},
        {"role": "user", "content": f"The following is a schema in JSON format with potentially irrelavant information: {input_schema}"},
        {"role": "user", "content": "Please extract only the relevant columns from the schema based on the task. The final output should be a schema in JSON format."},
    ]
)
print(completion.choices[0].message.content)

{
  "columns": [
    {
      "name": "age",
      "type": "INT",
      "constraints": ["NOT NULL", "CHECK (age >= 0)"],
      "description": "Age of the person"
    },
    {
      "name": "is_resident",
      "type": "BOOLEAN",
      "constraints": ["NOT NULL"],
      "description": "Whether the person is a resident of the state"
    },
    {
      "name": "is_unemployed",
      "type": "BOOLEAN",
      "constraints": ["NOT NULL"],
      "description": "Whether the person is unemployed"
    },
    {
      "name": "has_insufficient_means",
      "type": "BOOLEAN",
      "constraints": ["NOT NULL"],
      "description": "Whether the person has insufficient means"
    },
    {
      "name": "vision_impaired",
      "type": "BOOLEAN",
      "constraints": ["NOT NULL"],
      "description": "Whether the person has a vision impairment"
    },
    {
      "name": "is_widow_or_widower",
      "type": "BOOLEAN",
      "constraints": ["NOT NULL"],
      "description": "Whether the person is a wi

In [8]:
actual_output_schema = json.loads(completion.choices[0].message.content)
actual_output_schema == expected_output_schema

False

In [9]:
actual_cols = set([col['name'] for col in actual_output_schema['columns']])
expected_cols = set([col['name'] for col in expected_output_schema['columns']])

print(actual_cols - expected_cols)
print(expected_cols - actual_cols) # these are actually not needed so our expected output was wrong!

set()
{'last_name', 'first_name', 'id'}


## Final Schema Agent

In [10]:
import json
from openai import OpenAI

client = OpenAI()

def schema_agent(input_schema: dict, rules: str):
    """
    This is an AI agent that extracts the relevant columns from a schema based on a set of rules that define a task.

    Parameters:
        - input_schema: A dictionary containing the schema columns.
        - rules: A string containing the rules that define the task.
    
    Returns:
        - A dictionary containing the extracted columns from the schema.
    """
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You are a schema editing assistant, capable of extracting only the relevant columns based on a task. You only output in JSON format. Do not include other text or markdown tags in the output."},
            {"role": "user", "content": f"The relevant task is to determine the scheme of a person based on the following rules: {rules}"},
            {"role": "user", "content": f"The following is a schema in JSON format with potentially irrelavant information: {input_schema}"},
            {"role": "user", "content": "Please extract only the relevant columns from the schema based on the task. The final output should be a schema in JSON format."},
        ]
    )
    return json.loads(completion.choices[0].message.content)

In [11]:
schema_agent(input_schema, rules)

{'columns': [{'name': 'age',
   'type': 'INT',
   'constraints': ['NOT NULL', 'CHECK (age >= 0)'],
   'description': 'Age of the person'},
  {'name': 'is_resident',
   'type': 'BOOLEAN',
   'constraints': ['NOT NULL'],
   'description': 'Whether the person is a resident of the state'},
  {'name': 'is_unemployed',
   'type': 'BOOLEAN',
   'constraints': ['NOT NULL'],
   'description': 'Whether the person is unemployed'},
  {'name': 'has_insufficient_means',
   'type': 'BOOLEAN',
   'constraints': ['NOT NULL'],
   'description': 'Whether the person has insufficient means'},
  {'name': 'vision_impaired',
   'type': 'BOOLEAN',
   'constraints': ['NOT NULL'],
   'description': 'Whether the person has a vision impairment'},
  {'name': 'is_widow_or_widower',
   'type': 'BOOLEAN',
   'constraints': ['NOT NULL'],
   'description': 'Whether the person is a widow or widower'},
  {'name': 'is_not_cohabiting',
   'type': 'BOOLEAN',
   'constraints': ['NOT NULL'],
   'description': 'Whether the pers